In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time
from tqdm.auto import tqdm

In [ ]:
# Configuration
MODEL_NAME = 'prajjwal1/bert-small'
MAX_LEN = 96
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 5e-5
PATIENCE = 4

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} {'- ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else ''}")

Using device: cuda - Tesla T4


In [ ]:
# Load dataset
print("Loading dataset...")
df = pd.read_csv('/content/combined_student_queries_all.csv')

# Print dataset stats
print(f"Dataset shape: {df.shape}")
print(f"Class distribution:\n{df['Intent'].value_counts()}")

# Convert labels to numerical values
df['label'] = df['Intent'].map({'genuine': 0, 'manipulative': 1})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Initialize tokenizer
print(f"Loading {MODEL_NAME} tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading dataset...
Dataset shape: (3000, 2)
Class distribution:
Intent
manipulative    1511
genuine         1489
Name: count, dtype: int64
Loading prajjwal1/bert-small tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
# Dataset class
class QueryDataset(Dataset):
    def __init__(self, queries, labels, tokenizer, max_len):
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        query = str(self.queries[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create data loaders
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = QueryDataset(
        queries=df['Query'].values, #change to Query later
        labels=df['label'].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True if df is train_df else False
    )

In [ ]:
print("Creating data loaders...")
train_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)

Creating data loaders...


In [ ]:
# Initialize model
print(f"Loading {MODEL_NAME} model...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)
model = model.to(device)

Loading prajjwal1/bert-small model...


pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up optimizer and scheduler with weight decay
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Scheduler with warmup
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=total_steps
)

In [ ]:
import torch.cuda.amp as amp

# Initialize gradient scaler for mixed precision training
scaler = torch.amp.GradScaler('cuda')

In [ ]:
# Training function
def train_epoch(model, data_loader, optimizer, device, scheduler, epoch):
    model = model.train()
    losses = []
    correct_predictions = 0
    total_samples = 0

    # Progress bar with batch tracking
    progress_bar = tqdm(
        enumerate(data_loader),
        total=len(data_loader),
        desc=f"Epoch {epoch + 1}/{EPOCHS} [Train]",
        leave=True,
    )

    for batch_idx, batch in progress_bar:
        input_ids = batch['input_ids'].to(device, non_blocking=True)
        attention_mask = batch['attention_mask'].to(device, non_blocking=True)
        labels = batch['label'].to(device, non_blocking=True)

        # Clear gradients
        optimizer.zero_grad()

        # Mixed precision forward pass
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            logits = outputs.logits

        # Mixed precision backward pass
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_samples += labels.size(0)
        losses.append(loss.item())

        # Update progress less frequently to reduce overhead
        if batch_idx % 5 == 0 or batch_idx == len(data_loader) - 1:
            current_loss = np.mean(losses[-10:]) if losses else 0
            current_acc = (correct_predictions.double() / total_samples).item() if total_samples > 0 else 0

            progress_bar.set_postfix({
                "Loss": f"{current_loss:.4f}",
                "Acc": f"{current_acc:.4f}",
                "LR": f"{scheduler.get_last_lr()[0]:.6f}"
            })

    epoch_loss = np.mean(losses)
    epoch_acc = correct_predictions.double() / len(data_loader.dataset)

    return epoch_acc.item(), epoch_loss


In [ ]:
# Evaluation function
def eval_model(model, data_loader, device, epoch):
    model.eval()
    losses = []
    correct_predictions = 0

    # Progress bar for validation
    progress_bar = tqdm(
        enumerate(data_loader),
        total=len(data_loader),
        desc=f"Epoch {epoch + 1}/{EPOCHS} [Val]",
        leave=False,
    )

    with torch.no_grad():
        for batch_idx, batch in progress_bar:
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            labels = batch['label'].to(device, non_blocking=True)

            # Can still use mixed precision for inference
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            # Update less frequently
            if batch_idx % 10 == 0 or batch_idx == len(data_loader) - 1:
                progress_bar.set_postfix({
                    "Val Loss": f"{np.mean(losses[-10:]) if losses else 0:.4f}"
                })

    epoch_loss = np.mean(losses)
    epoch_acc = correct_predictions.double() / len(data_loader.dataset)

    return epoch_acc.item(), epoch_loss

In [ ]:
# Training loop with timing
best_accuracy = 0
best_val_loss = float('inf')
epochs_without_improvement = 0
total_start_time = time.time()

print(f"\n{'='*50}")
print("Starting training...")
print(f"{'='*50}\n")

for epoch in range(EPOCHS):
    epoch_start_time = time.time()

    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device, scheduler, epoch)
    val_acc, val_loss = eval_model(model, val_loader, device, epoch)

    epoch_time = time.time() - epoch_start_time

    print(
        f"\nEpoch {epoch + 1}/{EPOCHS} Summary:\n"
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}\n"
        f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}\n"
        f"Epoch Time: {epoch_time:.2f}s | Total Time: {(time.time() - total_start_time):.2f}s\n"
    )

    # Save model if validation accuracy improves
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model_acc.bin')
        print(f"✓ New best accuracy: {best_accuracy:.4f} - Model saved!\n")

    # Early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'best_model_loss.bin')
        print(f"✓ New best validation loss: {best_val_loss:.4f} - Model saved!\n")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= PATIENCE:
            print(f"❌ Early stopping at epoch {epoch+1}")
            break

# Final training statistics
total_time = time.time() - total_start_time
print(f"\n{'='*50}")
print(f"Training complete!")
print(f"Total Training Time: {total_time:.2f}s ({total_time/60:.2f} minutes)")
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation Loss: {best_val_loss:.4f}")
print(f"{'='*50}\n")


Starting training...



Epoch 1/20 [Train]:   0%|          | 0/150 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

Epoch 1/20 [Val]:   0%|          | 0/38 [00:00<?, ?it/s]


Epoch 1/20 Summary:
Train Loss: 0.3236 | Train Acc: 0.8417
Val Loss:   0.2747 | Val Acc:   0.8883
Epoch Time: 9.88s | Total Time: 9.88s

✓ New best accuracy: 0.8883 - Model saved!

✓ New best validation loss: 0.2747 - Model saved!



...

Epoch 5/20 [Val]:   0%|          | 0/38 [00:00<?, ?it/s]


Epoch 5/20 Summary:
Train Loss: 0.0242 | Train Acc: 0.9925
Val Loss:   0.4033 | Val Acc:   0.9267
Epoch Time: 5.69s | Total Time: 40.28s

✓ New best accuracy: 0.9267 - Model saved!



Epoch 6/20 [Train]:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 6/20 [Val]:   0%|          | 0/38 [00:00<?, ?it/s]


Epoch 6/20 Summary:
Train Loss: 0.0105 | Train Acc: 0.9979
Val Loss:   0.4756 | Val Acc:   0.9167
Epoch Time: 5.27s | Total Time: 45.79s

❌ Early stopping at epoch 6

Training complete!
Total Training Time: 45.79s (0.76 minutes)
Best Validation Accuracy: 0.9267
Best Validation Loss: 0.2375



In [ ]:
# Load best model
print("Loading best model for inference...")
model.load_state_dict(torch.load('best_model_acc.bin'))
model.eval()

Loading best model for inference...


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-1

In [ ]:
# Inference function
def predict(query, max_len=MAX_LEN):
    # Clean the text first
    cleaned_query = query

    encoding = tokenizer.encode_plus(
        cleaned_query,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    _, prediction = torch.max(logits, dim=1)

    intent = 'genuine' if prediction.item() == 0 else 'manipulative'
    confidence = probabilities[0][prediction.item()].item()

    return {
        'intent': intent,
        'confidence': confidence,
        'genuine_prob': probabilities[0][0].item(),
        'manipulative_prob': probabilities[0][1].item()
    }

In [ ]:
# Test prediction
test_queries = [
    "Why does my program keep crashing?",
    "Can you give me the answer to this question?",
    "What should I do if my program keeps crashing?",
    "Just give me the answer, I don't care about learning"
]

print("\nTesting model on example queries:")
for query in test_queries:
    result = predict(query)
    print(f"\nQuery: '{query}'")
    print(f"Prediction: {result['intent']} (Confidence: {result['confidence']:.4f})")
    print(f"Probabilities - Genuine: {result['genuine_prob']:.4f}, Manipulative: {result['manipulative_prob']:.4f}")


Testing model on example queries:

Query: 'Why does my program keep crashing?'
Prediction: genuine (Confidence: 0.9995)
Probabilities - Genuine: 0.9995, Manipulative: 0.0007

Query: 'Can you give me the answer to this question?'
Prediction: manipulative (Confidence: 1.0000)
Probabilities - Genuine: 0.0002, Manipulative: 1.0000

Query: 'What should I do if my program keeps crashing?'
Prediction: genuine (Confidence: 0.9980)
Probabilities - Genuine: 0.9980, Manipulative: 0.0020

Query: 'Just give me the answer, I don't care about learning'
Prediction: manipulative (Confidence: 0.9990)
Probabilities - Genuine: 0.0011, Manipulative: 0.9990


## Evaluation Metrics and Visualization
Now we will calculate accuracy, precision, recall, F1-score on the validation set, and plot the training loss and accuracy curves, as well as a bar chart of these metrics.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
 
# Get predictions and true labels for the validation set
def get_val_predictions(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

val_labels, val_preds = get_val_predictions(model, val_loader, device)

# Calculate metrics
accuracy = accuracy_score(val_labels, val_preds)
precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(val_labels, val_preds, target_names=['genuine', 'manipulative']))

In [ ]:
# Plot bar chart of metrics
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
metrics_values = [accuracy, precision, recall, f1]

plt.figure(figsize=(7,5))
sns.barplot(x=metrics_names, y=metrics_values, palette='viridis')
plt.ylim(0,1)
plt.title('Validation Metrics')
plt.ylabel('Score')
plt.show()

# Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(val_labels, val_preds, display_labels=['genuine', 'manipulative'], cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# If you have stored train_losses, val_losses, train_accuracies, val_accuracies during training, plot them:
try:
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(train_accuracies, label='Train Acc')
    plt.plot(val_accuracies, label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('Loss/accuracy curves not plotted. Make sure you have train_losses, val_losses, train_accuracies, val_accuracies lists.')